In [2]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, transpile, IBMQ
from qiskit.circuit import Parameter, ParameterVector
from qiskit.circuit.library import EfficientSU2, ZZFeatureMap, TwoLocal, NLocal
from qiskit.visualization import plot_histogram, plot_distribution

from qiskit.primitives import Sampler, Estimator

import numpy as np
import matplotlib.pyplot as plt

In [50]:
def conv_layer(qc:QuantumCircuit()) -> QuantumCircuit():
    start = 0 if qc.num_qubits == 2 else -1
    for i in range(start, qc.num_qubits-1):
        qc.rz(-np.pi / 2, i+1)
        qc.cx(i+1, i)
        qc.rz(para_vec[3*i], i)
        qc.ry(para_vec[(3*i)+1], i+1)
        qc.cx(i, i+1)
        qc.ry(para_vec[(3*i)+2], i+1)
        qc.cx(i+1, i)
        qc.rz(np.pi / 2, i)
        qc.barrier()
    qc_temp = qc
    return qc_temp

qc = QuantumCircuit(2)
para_vec = ParameterVector("θ", length=qc.num_qubits*3)
conv_layer(qc)
qc.draw(fold=130)

┌───┐┌──────────┐                 ┌───┐┌─────────┐ ░ 
q_0: ────────────┤ X ├┤ Rz(θ[0]) ├──■──────────────┤ X ├┤ Rz(π/2) ├─░─
     ┌──────────┐└─┬─┘├──────────┤┌─┴─┐┌──────────┐└─┬─┘└─────────┘ ░ 
q_1: ┤ Rz(-π/2) ├──■──┤ Ry(θ[1]) ├┤ X ├┤ Ry(θ[2]) ├──■──────────────░─
     └──────────┘     └──────────┘└───┘└──────────┘                 ░

In [49]:
def pool_layer(qc:QuantumCircuit()) -> QuantumCircuit():
    half = int(qc.num_qubits/2)
    for i in range(half):
        qc.rz(-np.pi / 2, i+half)
        qc.cx(i+half, i)
        qc.rz(para_vec[3*i], i)
        qc.ry(para_vec[(3*i)+half], i+half)
        qc.cx(i, i+half)
        qc.ry(para_vec[(3*i)+2], i+half)
        qc.barrier()
    qc_temp = qc
    return qc_temp

qc = QuantumCircuit(4)
para_vec = ParameterVector("θ", length=qc.num_qubits*3)
pool_layer(qc)
qc.draw(fold=130)

┌───┐┌──────────┐                  ░                                                ░ 
q_0: ────────────┤ X ├┤ Rz(θ[0]) ├──■───────────────░────────────────────────────────────────────────░─
                 └─┬─┘└──────────┘  │               ░             ┌───┐┌──────────┐                  ░ 
q_1: ──────────────┼────────────────┼───────────────░─────────────┤ X ├┤ Rz(θ[3]) ├──■───────────────░─
     ┌──────────┐  │  ┌──────────┐┌─┴─┐┌──────────┐ ░             └─┬─┘└──────────┘  │               ░ 
q_2: ┤ Rz(-π/2) ├──■──┤ Ry(θ[2]) ├┤ X ├┤ Ry(θ[2]) ├─░───────────────┼────────────────┼───────────────░─
     └──────────┘     └──────────┘└───┘└──────────┘ ░ ┌──────────┐  │  ┌──────────┐┌─┴─┐┌──────────┐ ░ 
q_3: ───────────────────────────────────────────────░─┤ Rz(-π/2) ├──■──┤ Ry(θ[5]) ├┤ X ├┤ Ry(θ[5]) ├─░─
                                                    ░ └──────────┘     └──────────┘└───┘└──────────┘ ░